In [2]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

In [3]:
class FinanceChartVisualizer:
    def __init__(self, csv_file_path, date_col="DATE", open_col="OPEN PRICE", 
                 high_col="HIGH PRICE", low_col="LOW PRICE", close_col="CLOSE PRICE", 
                 volume_col="Volume"):
        """
        Initialize the visualizer with CSV file and column mappings
        """
        self.csv_file_path = csv_file_path
        self.date_col = date_col.strip()
        self.open_col = open_col.strip()
        self.high_col = high_col.strip()
        self.low_col = low_col.strip()
        self.close_col = close_col.strip()
        self.volume_col = volume_col.strip()
        self.df = None
        
    def load_data(self):
        """Load and preprocess the CSV data"""
        # Read CSV and clean column names
        self.df = pd.read_csv(self.csv_file_path)
        self.df.columns = self.df.columns.str.strip()
        
        # Convert date column to datetime
        self.df[self.date_col] = pd.to_datetime(self.df[self.date_col], format='%d-%b-%Y')
        
        # Convert numeric columns, handling commas in numbers
        numeric_cols = [self.open_col, self.high_col, self.low_col, self.close_col]
        for col in numeric_cols:
            if col in self.df.columns:
                self.df[col] = pd.to_numeric(self.df[col], errors='coerce')
        
        # Handle volume with commas
        if self.volume_col in self.df.columns:
            self.df[self.volume_col] = self.df[self.volume_col].astype(str).str.replace(',', '')
            self.df[self.volume_col] = pd.to_numeric(self.df[self.volume_col], errors='coerce')
        
        # Sort by date
        self.df = self.df.sort_values(self.date_col).reset_index(drop=True)
        
        # Remove rows with missing OHLC data
        self.df = self.df.dropna(subset=[self.open_col, self.high_col, self.low_col, self.close_col])
        
        print(f"Data loaded: {len(self.df)} records from {self.df[self.date_col].min()} to {self.df[self.date_col].max()}")
        return self.df
    
    def calculate_indicators(self, sma_periods=[20, 50], ema_periods=[12, 26], 
                           rsi_period=14, bb_period=20, bb_std=2):
        """Calculate technical indicators"""
        if self.df is None:
            self.load_data()
        
        # Simple Moving Averages
        for period in sma_periods:
            self.df[f'SMA_{period}'] = self.df[self.close_col].rolling(window=period).mean()
        
        # Exponential Moving Averages
        for period in ema_periods:
            self.df[f'EMA_{period}'] = self.df[self.close_col].ewm(span=period).mean()
        
        # RSI (Relative Strength Index)
        delta = self.df[self.close_col].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=rsi_period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=rsi_period).mean()
        rs = gain / loss
        self.df['RSI'] = 100 - (100 / (1 + rs))
        
        # MACD
        exp1 = self.df[self.close_col].ewm(span=12).mean()
        exp2 = self.df[self.close_col].ewm(span=26).mean()
        self.df['MACD'] = exp1 - exp2
        self.df['MACD_signal'] = self.df['MACD'].ewm(span=9).mean()
        self.df['MACD_histogram'] = self.df['MACD'] - self.df['MACD_signal']
        
        # Bollinger Bands
        self.df['BB_middle'] = self.df[self.close_col].rolling(window=bb_period).mean()
        bb_std_dev = self.df[self.close_col].rolling(window=bb_period).std()
        self.df['BB_upper'] = self.df['BB_middle'] + (bb_std_dev * bb_std)
        self.df['BB_lower'] = self.df['BB_middle'] - (bb_std_dev * bb_std)
        
        # Stochastic Oscillator
        low_14 = self.df[self.low_col].rolling(window=14).min()
        high_14 = self.df[self.high_col].rolling(window=14).max()
        self.df['%K'] = 100 * ((self.df[self.close_col] - low_14) / (high_14 - low_14))
        self.df['%D'] = self.df['%K'].rolling(window=3).mean()
        
        print("Technical indicators calculated")
        
    def create_advanced_chart(self, title="Advanced Finance Chart", output_file="finance_chart.html",
                            show_volume=True, show_rsi=True, show_macd=True, show_stoch=True):
        """Create comprehensive financial chart with multiple indicators"""
        
        if self.df is None:
            self.load_data()
        
        self.calculate_indicators()
        
        # Define subplot structure
        subplot_titles = [title]
        rows = 1
        row_heights = [0.6]
        
        if show_volume:
            subplot_titles.append("Volume")
            rows += 1
            row_heights.append(0.15)
        
        if show_rsi:
            subplot_titles.append("RSI (14)")
            rows += 1
            row_heights.append(0.1)
        
        if show_macd:
            subplot_titles.append("MACD")
            rows += 1
            row_heights.append(0.1)
            
        if show_stoch:
            subplot_titles.append("Stochastic")
            rows += 1
            row_heights.append(0.05)
        
        # Create subplots
        fig = make_subplots(
            rows=rows, cols=1, 
            shared_xaxes=True,
            vertical_spacing=0.02,
            subplot_titles=subplot_titles,
            row_heights=row_heights,
            specs=[[{"secondary_y": False}] for _ in range(rows)]
        )
        
        current_row = 1
        
        # Main candlestick chart
        fig.add_trace(
            go.Candlestick(
                x=self.df[self.date_col],
                open=self.df[self.open_col],
                high=self.df[self.high_col],
                low=self.df[self.low_col],
                close=self.df[self.close_col],
                name="OHLC",
                showlegend=False
            ),
            row=current_row, col=1
        )
        
        # Add moving averages
        colors = ['orange', 'purple', 'brown', 'pink']
        ma_columns = [col for col in self.df.columns if col.startswith(('SMA_', 'EMA_'))]
        for i, ma_col in enumerate(ma_columns):
            fig.add_trace(
                go.Scatter(
                    x=self.df[self.date_col],
                    y=self.df[ma_col],
                    mode='lines',
                    name=ma_col,
                    line=dict(color=colors[i % len(colors)], width=1),
                    showlegend=True
                ),
                row=current_row, col=1
            )
        
        # Add Bollinger Bands
        if 'BB_upper' in self.df.columns:
            fig.add_trace(
                go.Scatter(
                    x=self.df[self.date_col],
                    y=self.df['BB_upper'],
                    mode='lines',
                    name='BB Upper',
                    line=dict(color='gray', width=1, dash='dash'),
                    showlegend=False
                ),
                row=current_row, col=1
            )
            
            fig.add_trace(
                go.Scatter(
                    x=self.df[self.date_col],
                    y=self.df['BB_lower'],
                    mode='lines',
                    name='BB Lower',
                    line=dict(color='gray', width=1, dash='dash'),
                    fill='tonexty',
                    fillcolor='rgba(128,128,128,0.1)',
                    showlegend=False
                ),
                row=current_row, col=1
            )
        
        current_row += 1
        
        # Volume chart
        if show_volume and self.volume_col in self.df.columns:
            colors_vol = ['green' if close >= open else 'red' 
                         for close, open in zip(self.df[self.close_col], self.df[self.open_col])]
            
            fig.add_trace(
                go.Bar(
                    x=self.df[self.date_col],
                    y=self.df[self.volume_col],
                    name="Volume",
                    marker_color=colors_vol,
                    showlegend=False
                ),
                row=current_row, col=1
            )
            current_row += 1
        
        # RSI
        if show_rsi and 'RSI' in self.df.columns:
            fig.add_trace(
                go.Scatter(
                    x=self.df[self.date_col],
                    y=self.df['RSI'],
                    mode='lines',
                    name='RSI',
                    line=dict(color='purple'),
                    showlegend=False
                ),
                row=current_row, col=1
            )
            
            # Add RSI levels
            fig.add_hline(y=70, line_dash="dash", line_color="red", 
                         annotation_text="Overbought", row=current_row, col=1)
            fig.add_hline(y=30, line_dash="dash", line_color="green", 
                         annotation_text="Oversold", row=current_row, col=1)
            fig.add_hline(y=50, line_dash="dot", line_color="gray", row=current_row, col=1)
            
            current_row += 1
        
        # MACD
        if show_macd and 'MACD' in self.df.columns:
            fig.add_trace(
                go.Scatter(
                    x=self.df[self.date_col],
                    y=self.df['MACD'],
                    mode='lines',
                    name='MACD',
                    line=dict(color='blue'),
                    showlegend=False
                ),
                row=current_row, col=1
            )
            
            fig.add_trace(
                go.Scatter(
                    x=self.df[self.date_col],
                    y=self.df['MACD_signal'],
                    mode='lines',
                    name='Signal',
                    line=dict(color='red'),
                    showlegend=False
                ),
                row=current_row, col=1
            )
            
            # MACD Histogram
            colors_macd = ['green' if val >= 0 else 'red' for val in self.df['MACD_histogram']]
            fig.add_trace(
                go.Bar(
                    x=self.df[self.date_col],
                    y=self.df['MACD_histogram'],
                    name='MACD Histogram',
                    marker_color=colors_macd,
                    showlegend=False
                ),
                row=current_row, col=1
            )
            
            current_row += 1
        
        # Stochastic
        if show_stoch and '%K' in self.df.columns:
            fig.add_trace(
                go.Scatter(
                    x=self.df[self.date_col],
                    y=self.df['%K'],
                    mode='lines',
                    name='%K',
                    line=dict(color='blue'),
                    showlegend=False
                ),
                row=current_row, col=1
            )
            
            fig.add_trace(
                go.Scatter(
                    x=self.df[self.date_col],
                    y=self.df['%D'],
                    mode='lines',
                    name='%D',
                    line=dict(color='red'),
                    showlegend=False
                ),
                row=current_row, col=1
            )
            
            fig.add_hline(y=80, line_dash="dash", line_color="red", row=current_row, col=1)
            fig.add_hline(y=20, line_dash="dash", line_color="green", row=current_row, col=1)
        
        # Update layout
        fig.update_layout(
            title=f"{title} - Advanced Technical Analysis",
            xaxis_rangeslider_visible=False,
            height=800,
            showlegend=True,
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1
            )
        )
        
        # Update x-axis labels
        fig.update_xaxes(title_text="Date", row=rows, col=1)
        
        # Save to HTML
        fig.write_html(output_file)
        print(f"Advanced chart saved to {output_file}")
        
        return fig


In [5]:
# Usage example for your USD-INR data
def create_usdinr_chart():
    # Initialize visualizer
    visualizer = FinanceChartVisualizer(
        csv_file_path="Data/Quote-CD-USDINR-15-09-2024-to-15-09-2025.csv",
        date_col="DATE",
        open_col="OPEN PRICE", 
        high_col="HIGH PRICE",
        low_col="LOW PRICE", 
        close_col="CLOSE PRICE",
        volume_col="Volume"
    )
    
    # Create chart
    fig = visualizer.create_advanced_chart(
        title="USD-INR Futures Analysis",
        output_file="usdinr_advanced_chart.html",
        show_volume=True,
        show_rsi=True, 
        show_macd=True,
        show_stoch=True
    )
    
    return fig

In [6]:
# Create the chart
fig = create_usdinr_chart()

# Display in Jupyter (optional)
fig.show()

Data loaded: 92 records from 2024-09-30 00:00:00 to 2025-09-12 00:00:00
Technical indicators calculated
Advanced chart saved to usdinr_advanced_chart.html
